In [1]:
'''
特殊分类视频信息分析 AI换脸
http://www.h4ck.org.cn
by obaby
obaby@mars
email:root@obaby.org.cn
date: 2020.09.11
'''

from pyspark.sql.functions import col
import altair as alt

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
csv = spark.read.option("header",True).csv("hdfs://localhost:9000/data2/porn_data_movie.csv")
csv.printSchema()

root
 |-- id: string (nullable = true)
 |-- create: string (nullable = true)
 |-- update: string (nullable = true)
 |-- name: string (nullable = true)
 |-- describe: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- play_count: string (nullable = true)
 |-- good_count: string (nullable = true)
 |-- bad_count: string (nullable = true)
 |-- link_count: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- porn_site_id: string (nullable = true)
 |-- uploader_id: string (nullable = true)
 |-- producer: string (nullable = true)



In [3]:
csv.select('name', 'describe', 'uploader_id').show()

+------------------------+------------------------+-----------+
|                    name|                describe|uploader_id|
+------------------------+------------------------+-----------+
|    美女学生考试时被中出|                    null|          1|
|      无套中出内射（上）|            无套中出内射|          2|
|      无套中出内射（下）|            无套中出内射|          2|
|                极品嫩妹|                    null|          3|
|                骚妹妹 7|                    null|          3|
|漂亮美女完美身材甜美声音|漂亮美女完美身材甜美声音|          4|
|          jk捆绑萝莉束缚|                    null|          1|
|            最纯萝莉性爱|                    null|          1|
|          整容脸制服美女|                    null|          1|
|             星 调教萝莉|                    null|          5|
|        大一学生寝室自慰|                    null|          6|
|    超性感美女给你打飞机|                    null|          6|
|      超可爱萝莉下海资源|                    null|          6|
|      迷倒女儿然后慢慢操|                    null|          6|
|   精灵做爱高潮–影视剪辑|                    null|  

In [4]:
# 分类信息读取 
category_csv = spark.read.option("header",True).csv("hdfs://localhost:9000/data2/porn_data_category.csv")

In [5]:
category_csv.printSchema()

root
 |-- id: string (nullable = true)
 |-- create: string (nullable = true)
 |-- update: string (nullable = true)
 |-- name: string (nullable = true)
 |-- key: string (nullable = true)
 |-- describe: string (nullable = true)



In [7]:
movie_csv = csv.withColumnRenamed('name','movie_name')
movie_csv.select('movie_name', 'category_id', 'id').show()

+---+--------------------+--------------------+------------------------+------------------------+---------+------------+----------+----------+---------+----------+-------------+-----------+-----------+------------+-----------+--------+
| id|              create|              update|              movie_name|                describe|source_id|publish_time|play_count|good_count|bad_count|link_count|comment_count|designation|category_id|porn_site_id|uploader_id|producer|
+---+--------------------+--------------------+------------------------+------------------------+---------+------------+----------+----------+---------+----------+-------------+-----------+-----------+------------+-----------+--------+
|  1|7/5/2020 05:20:14...|7/5/2020 05:20:14...|    美女学生考试时被中出|                    null|    76009|  1588819417|      2566|        14|        2|        48|            0|       null|          1|           1|          1|    null|
|  2|7/5/2020 05:21:04...|7/5/2020 05:21:04...|      无套中出内射（上）|   

In [8]:
movie_cat_rdd = movie_csv.select('movie_name','category_id').join(category_csv, movie_csv.category_id == category_csv.id, "inner")



In [9]:
movie_cat_rdd.select('movie_name', 'name', 'id').show()

+------------------------+------+---+
|              movie_name|  name| id|
+------------------------+------+---+
|    美女学生考试时被中出|萝莉系|  1|
|      无套中出内射（上）|萝莉系|  1|
|      无套中出内射（下）|萝莉系|  1|
|                极品嫩妹|萝莉系|  1|
|                骚妹妹 7|萝莉系|  1|
|漂亮美女完美身材甜美声音|萝莉系|  1|
|          jk捆绑萝莉束缚|萝莉系|  1|
|            最纯萝莉性爱|萝莉系|  1|
|          整容脸制服美女|萝莉系|  1|
|             星 调教萝莉|萝莉系|  1|
|        大一学生寝室自慰|萝莉系|  1|
|    超性感美女给你打飞机|萝莉系|  1|
|      超可爱萝莉下海资源|萝莉系|  1|
|      迷倒女儿然后慢慢操|萝莉系|  1|
|   精灵做爱高潮–影视剪辑|萝莉系|  1|
|        小萝莉被插到大叫|萝莉系|  1|
|    睡醒和白嫩女友来一发|萝莉系|  1|
|  双马尾萝莉甜美自慰诱惑|萝莉系|  1|
|        调教双马尾小萝莉|萝莉系|  1|
|    黑丝小萝莉最喜欢玩具|萝莉系|  1|
+------------------------+------+---+
only showing top 20 rows



In [17]:
import jieba.posseg as psg

def get_person_name_from_line(line):
    words = psg.cut(str(line))
    l =[]
    for x in words:
        if x.flag =='nr':
            l.append(x.word)
        # print(x.word, x.flag)
    return l
    

In [21]:
# 测试文字 http://m.lewenb.com/wapbook-1315-235763/
# document.body.innerText
w2 = get_person_name_from_line('朱宜锐的一只大手，隔着一层绵薄滑软的抚握住范冰冰那一只弹挺柔软的，他的手轻而不急地揉捏着……手掌间传来一阵坚挺结实、柔软无比而又充满弹性的美妙肉感，令他血脉贲张。看见澜那线条优美的秀丽桃腮上，一抹醉人的晕红正逐渐蔓衍到她那美艳动人的绝色娇靥上，他不由得色心一荡，他的手指逐渐收拢，轻轻地用两根手指轻抚范冰冰下那傲挺的峰顶，打着圈的轻抚揉压，找到那一粒娇小玲珑的挺突之巅的。他两根手指轻轻地夹住范冰冰那娇软柔小的，温柔而有技巧地一阵揉搓、轻捏。范冰冰被那从敏感地带的玉上传来的异样的感觉弄得浑身如被虫噬。一想到就连自己平常一个人都不好意思久看，不敢轻触的娇小被这样一个陌生而又恶心的男人（至少在她看来是这样）肆意揉搓轻抚，芳心不觉又感到万分的羞涩和莫名的刺激。朱宜锐一面揉捏着范冰冰那娇小的，一面继续在她的深处抽动着……')
print(w2)

['朱宜锐', '范冰冰', '贲', '张', '范冰冰', '范冰冰', '范冰冰', '朱宜锐', '范冰冰']


In [25]:
category_csv.filter(col('name')== 'AI换脸').show()

+---+--------------------+--------------------+------+----------+--------+
| id|              create|              update|  name|       key|describe|
+---+--------------------+--------------------+------+----------+--------+
|  9|7/5/2020 06:51:12...|27/8/2020 03:32:4...|AI换脸|aihuanlian|    null|
+---+--------------------+--------------------+------+----------+--------+



In [27]:
movie_cat_rdd.show()

+------------------------+-----------+---+--------------------+--------------------+------+-------+--------+
|              movie_name|category_id| id|              create|              update|  name|    key|describe|
+------------------------+-----------+---+--------------------+--------------------+------+-------+--------+
|    美女学生考试时被中出|          1|  1|7/5/2020 05:20:14...|27/8/2020 03:31:2...|萝莉系|luolixi|    null|
|      无套中出内射（上）|          1|  1|7/5/2020 05:20:14...|27/8/2020 03:31:2...|萝莉系|luolixi|    null|
|      无套中出内射（下）|          1|  1|7/5/2020 05:20:14...|27/8/2020 03:31:2...|萝莉系|luolixi|    null|
|                极品嫩妹|          1|  1|7/5/2020 05:20:14...|27/8/2020 03:31:2...|萝莉系|luolixi|    null|
|                骚妹妹 7|          1|  1|7/5/2020 05:20:14...|27/8/2020 03:31:2...|萝莉系|luolixi|    null|
|漂亮美女完美身材甜美声音|          1|  1|7/5/2020 05:20:14...|27/8/2020 03:31:2...|萝莉系|luolixi|    null|
|          jk捆绑萝莉束缚|          1|  1|7/5/2020 05:20:14...|27/8/2020 03:31:2...|萝莉系|lu

In [29]:
# 只分析AI换脸分类下的数据，虽然会有一部分遗漏，如果全部数据都进行解析会引入大量的无关数据（主要由于分词精度不够）
name_rdd = movie_cat_rdd.filter(col('category_id')=='9').select('movie_name').rdd

In [30]:
name_word_rdd = name_rdd.flatMap(get_person_name_from_line)

In [31]:
name_word_rdd.take(20)

['林允儿',
 '杨幂',
 '祖儿',
 '刘亦菲',
 '明星',
 '陈乔恩',
 '容祖儿',
 '林允儿',
 '明星',
 '宋',
 '王鸥',
 '迪丽',
 '刘亦菲',
 '明星',
 '迪丽',
 '陈乔恩',
 '刘亦菲',
 '刘亦菲',
 '迪丽',
 '陈乔恩']

In [32]:
# map reduce 进行词频统计
name_counts_rdd = name_word_rdd.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)


In [33]:
print('姓名统计一共:', str(name_counts_rdd.count()))

姓名统计一共: 100


In [34]:
name_counts_rdd.sortBy(lambda a: a[1],ascending=False).collect()[:20]

[('明星', 33),
 ('杨幂', 27),
 ('迪丽', 27),
 ('刘亦菲', 17),
 ('高清', 14),
 ('杨颖', 13),
 ('林允儿', 10),
 ('鞠', 8),
 ('杨', 8),
 ('白虎', 6),
 ('刘诗诗', 6),
 ('宋雨琦', 6),
 ('佟丽娅', 6),
 ('宋', 5),
 ('陈乔恩', 5),
 ('爽', 5),
 ('范冰冰', 4),
 ('刘涛', 4),
 ('林志玲', 4),
 ('宋祖儿', 3)]

In [35]:
name_counts_df = name_counts_rdd.toDF().toPandas()


In [36]:
name_counts_df.head()

,_1,_2
0,杨幂,27
1,祖儿,1
2,明星,33
3,容祖儿,2
4,宋,5


In [38]:
# 视频标签数量展示
bars=alt.Chart(name_counts_df).mark_bar().encode(
    x=alt.X('_2', title='视频数量'),
    y=alt.Y('_1', title='明星姓名', sort='-x')
)
text = bars.mark_text(align='left', baseline='middle', dx=3).encode(text='_2')
(bars + text).properties(height=1400, width=800)

alt.LayerChart(...)

In [42]:
# 迪丽-> 迪丽热巴
# 对于分词之后的姓名可以做进一步的解析，通过人工分析可以修正解析到的错误姓名。
# https://intellipaat.com/community/11489/filter-spark-dataframe-on-string-contains
movie_cat_rdd.filter(col('movie_name').contains('迪丽')).select('movie_name').show()

+-----------------------------------+
|                         movie_name|
+-----------------------------------+
|重金购买长相酷似女星迪丽XX的性感...|
|           迪丽热巴刘亦菲等明星合集|
|             迪丽热巴换上盛装自己嗨|
|               Al迪丽热巴大长腿中出|
|                       低配迪丽热巴|
|                    AI换脸 迪丽热巴|
|                           迪丽热巴|
|            AI换脸 迪丽热巴大战炮机|
|                 AI迪丽热巴自慰绝美|
|     [AI换脸] 迪丽热巴！演艺圈淫...|
|                 [AI换脸] 迪丽热巴2|
|【明星淫梦】人工智能AI让女神们下...|
|                 【AI换脸】迪丽热巴|
|            国产，酷似迪丽热巴 爆菊|
|                    AI换脸 迪丽热巴|
|                 [AI换脸] 迪丽热巴3|
|           迪丽热巴唯美身材激情自慰|
|                           迪丽热巴|
|                           迪丽热巴|
|                  迪丽热巴 智能换脸|
+-----------------------------------+
only showing top 20 rows



In [45]:
movie_cat_rdd.filter(col('movie_name').contains('鞠')).select('movie_name').show()

+-------------------------------+
|                     movie_name|
+-------------------------------+
|             宋轶鞠婧祎模特洗礼|
|               鞠婧祎和一群男人|
|【9uu神手】AI换脸之鞠婧祎 骚...|
|                  AI换脸 鞠婧炜|
|          【AI】鞠婧祎 短发空乘|
|         Al换脸鞠婧祎双胞胎服侍|
|                         鞠婧祎|
|                  AI换脸鞠婧祎 |
|          原创ai-宋轶鞠婧祎长腿|
|         ai换脸鞠婧祎干朋友妻子|
|           AI换脸女神鞠婧祎合集|
|           AI原创鞠婧祎激情颜射|
+-------------------------------+



In [ ]:
# 除了部分人名解析有问题，另外还有一部分数据原本不是人名，解析成了人名。例如玉足、痴汉等等~~
# 通过视频数量侧面也反映出了女明星在宅男中的热度？或者流行程度？